# 01_Crop_NIFTI_all.ipynb

This script crop all the NIFTI images listed in the crop array (`CropArray_CT_NIFTI_full.csv`) and generate the cropped NIFTI files. 

The crop array gives the pixel values (cropped volume) a size of $120\times120\times n$ (for one adrenal gland). Combining the left and right adrenals in one volume would gives a volume of $240\times120\times n$. Where $n$ is the number of axial slices that covered the whole (both left and right) adrenal glands.

All generate the cropped NIFTI files are saved in the folder: `CT_data/CT_NIFTI_cropped_combineLR`.

Required files:
- `DICOM_Data.csv`
- `CropArray_CT_NIFTI_full.csv`

Generate:
- Cropped NIFTI files

In [ ]:
import sys
repo_dir = <PATH TO THIS REPO> # local path to this repo.
sys.path.insert(0, repo_dir)

In [1]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm

# utilitiy files located in the src folder
from src.util_image import crop_3dimage, save_crop_3d

In [2]:
# locating the folder paths used in this script.
work_dir = <PATH TO project_dir> # path to the directory of the current project (project master path)
data_folder_path = work_dir + "CT_data/"
CT_NIFTI_full_path = data_folder_path + "CT_NIFTI_full/"
targetpath = data_folder_path + "CT_NIFTI_cropped_combineLR/"
df_crop = pd.read_csv(CT_NIFTI_full_path+"CropArray_CT_NIFTI_fullimage.csv",index_col=0)
df_dicom = pd.read_csv(data_folder_path+"DICOM_Data.csv")

In [3]:
# get the list of files to process for the cropping
process_list = list(set(df_crop["filename_nii"]))

### Crop the NIFTI images included in the process_list

Cropped NIFTI images to save in `targetpath`.

In [ ]:
for filename in tqdm(process_list):
    # Get cropped image according to the set boundaries
    df_current = df_crop[df_crop["filename_nii"]==filename]
    patient_state = df_dicom[df_dicom["filename_3d"]==filename.replace('.nii','')].normal_patient.array[0]
    filepath = CT_NIFTI_full_path + patient_state + "/" + filename
    for side in df_current.adrenal_LR:
        df_current_LR = df_current[df_current["adrenal_LR"]==side]
        crop_x = [df_current_LR.crop_x_min.values[0], df_current_LR.crop_x_max.values[0]]
        crop_y = [df_current_LR.crop_y_min.values[0], df_current_LR.crop_y_max.values[0]]
        crop_z = [df_current_LR.crop_z_min.values[0], df_current_LR.crop_z_max.values[0]]
        crop_array = [crop_x,crop_y,crop_z]
        if side == "L":
            image_3d_cropped_L, aff_3d, header_3d = crop_3dimage(filepath, crop_array)
        else:
            image_3d_cropped_R, aff_3d, header_3d = crop_3dimage(filepath, crop_array)
    try:
        image_3d_cropped = np.concatenate([image_3d_cropped_R, image_3d_cropped_L])
        save_crop_3d(filename, targetpath, image_3d_cropped, aff_3d, header_3d)
    except:
        print(filename)